In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [21]:
import datetime

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, concatenate, Multiply, Add

In [3]:
import sys
sys.path.append('/Users/dmitry/Desktop/Thesis/topics_ds/models')
sys.path.append('/Users/dmitry/Desktop/Thesis/topics_ds/data')

In [4]:
from models import models as models
from data import data as data

Using TensorFlow backend.


In [5]:
x_img, x_txt, y = data.get_unpacked_data()

In [13]:
x_img_train, x_img_test, x_txt_train, x_txt_test, y_train, y_test = train_test_split(
    x_img, 
    x_txt, 
    y, 
    test_size=0.2, 
    random_state=42,
    stratify=y
)

x_img_train, x_img_val, x_txt_train, x_txt_val, y_train, y_val = train_test_split(
    x_img_train,
    x_txt_train,
    y_train,
    test_size=0.2,
    random_state=42,
    stratify=y_train
)

In [14]:
zero_indicies = np.where(~x_txt_train.any(axis=1))[0]

In [18]:
non_zero_indicies = np.where(x_txt_train.any(axis=1))[0]

In [19]:
non_zero_indicies.shape

(118681,)

In [15]:
zero_indicies.shape

(17216,)

In [16]:
IMG_LEN = 1024
TXT_LEN = 300
N_CLASSES = 50

In [23]:
def get_model_img_to_txt():
    inp_img = Input(shape=(IMG_LEN,))
    
    x = Dense(512, activation='relu')(inp_img)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    out_txt = Dense(TXT_LEN, activation='sigmoid')(x)
    model = Model(inputs=inp_img, outputs=out_txt)
    model.compile(optimizer='adam', loss='mse')
    return model

In [41]:
def get_model_img_to_txt_1():
    inp_img = Input(shape=(IMG_LEN,))
    
    x = Dense(512, activation='relu')(inp_img)
    x = Dropout(0.25)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.25)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.25)(x)
    
    out_txt = Dense(TXT_LEN, activation='sigmoid')(x)
    model = Model(inputs=inp_img, outputs=out_txt)
    model.compile(optimizer='adam', loss='mse')
    return model

In [24]:
model_img_to_txt = get_model_img_to_txt()

log_dir="logs/img_to_txt/first" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

model_img_to_txt.fit(
    x_img_train[non_zero_indicies],
    x_txt_train[non_zero_indicies],
    validation_split=0.2,
    epochs=20,
    batch_size=2048,
    callbacks=[tensorboard_callback]
)

Train on 94944 samples, validate on 23737 samples
Epoch 1/20
94944/94944 [==============================] - 19s 199us/sample - loss: 0.0352 - val_loss: 0.0062
Epoch 2/20
94944/94944 [==============================] - 9s 91us/sample - loss: 0.0041 - val_loss: 0.0034
Epoch 3/20
94944/94944 [==============================] - 12s 123us/sample - loss: 0.0034 - val_loss: 0.0033
Epoch 4/20
94944/94944 [==============================] - 9s 99us/sample - loss: 0.0034 - val_loss: 0.0033
Epoch 5/20
94944/94944 [==============================] - 10s 100us/sample - loss: 0.0033 - val_loss: 0.0033
Epoch 6/20
94944/94944 [==============================] - 11s 114us/sample - loss: 0.0033 - val_loss: 0.0033
Epoch 7/20
94944/94944 [==============================] - 11s 112us/sample - loss: 0.0033 - val_loss: 0.0033
Epoch 8/20
94944/94944 [==============================] - 10s 109us/sample - loss: 0.0033 - val_loss: 0.0033
Epoch 9/20
94944/94944 [==============================] - 11s 111us/sample - loss:

In [42]:
model_img_to_txt_1 = get_model_img_to_txt_1()

log_dir="logs/img_to_txt/model_1_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

model_img_to_txt_1.fit(
    x_img_train[non_zero_indicies],
    x_txt_train[non_zero_indicies],
    validation_split=0.2,
    epochs=20,
    batch_size=2048,
    callbacks=[tensorboard_callback]
)

Train on 94944 samples, validate on 23737 samples
Epoch 1/20
94944/94944 [==============================] - 22s 232us/sample - loss: 0.0271 - val_loss: 0.0033
Epoch 2/20
94944/94944 [==============================] - 16s 173us/sample - loss: 0.0033 - val_loss: 0.0033
Epoch 3/20
94944/94944 [==============================] - 12s 125us/sample - loss: 0.0033 - val_loss: 0.0033
Epoch 4/20
94944/94944 [==============================] - 11s 116us/sample - loss: 0.0033 - val_loss: 0.0033
Epoch 5/20
94944/94944 [==============================] - 12s 131us/sample - loss: 0.0033 - val_loss: 0.0033
Epoch 6/20
94944/94944 [==============================] - 11s 116us/sample - loss: 0.0033 - val_loss: 0.0033
Epoch 7/20
94944/94944 [==============================] - 11s 114us/sample - loss: 0.0033 - val_loss: 0.0033
Epoch 8/20
94944/94944 [==============================] - 12s 123us/sample - loss: 0.0033 - val_loss: 0.0033
Epoch 9/20
94944/94944 [==============================] - 11s 115us/sample - l

In [ ]:
x_txt_train_modified = x_txt_train.copy()
x_txt_train_modified[zero_indicies] = model_img_to_txt.predict(x_img_train[zero_indicies])

In [27]:
model_trivial = models.get_model_trivial()

log_dir="logs/zeros_replaced/trivial_1_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

model_trivial.fit(
    [x_img_train, x_txt_train_modified],
    y_train,
    validation_data=([x_img_val, x_txt_val], y_val),
    epochs=20,
    callbacks=[tensorboard_callback]
)

Train on 135897 samples, validate on 33975 samples
Epoch 1/20
135897/135897 [==============================] - 16s 118us/sample - loss: 2.2202 - accuracy: 0.4284 - val_loss: 1.7005 - val_accuracy: 0.5430
Epoch 2/20
135897/135897 [==============================] - 14s 102us/sample - loss: 1.8706 - accuracy: 0.5048 - val_loss: 1.6152 - val_accuracy: 0.5649
Epoch 3/20
135897/135897 [==============================] - 13s 93us/sample - loss: 1.7882 - accuracy: 0.5250 - val_loss: 1.5560 - val_accuracy: 0.5764
Epoch 4/20
135897/135897 [==============================] - 12s 87us/sample - loss: 1.7500 - accuracy: 0.5352 - val_loss: 1.5375 - val_accuracy: 0.5812
Epoch 5/20
135897/135897 [==============================] - 12s 90us/sample - loss: 1.7221 - accuracy: 0.5420 - val_loss: 1.5375 - val_accuracy: 0.5845
Epoch 6/20
135897/135897 [==============================] - 12s 91us/sample - loss: 1.6992 - accuracy: 0.5481 - val_loss: 1.5291 - val_accuracy: 0.5872
Epoch 7/20
135897/135897 [=========

In [28]:
model_wide_2048 = models.get_model_custom_lr_wide()
log_dir_wide_2048 ="logs/zeros_replaced/wide_2048_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback_wide_2048 = TensorBoard(log_dir=log_dir_wide_2048, histogram_freq=1)

model_wide_2048.fit(
    [x_img_train, x_txt_train_modified],
    y_train,
    validation_data=([x_img_val, x_txt_val], y_val),
    epochs=20,
    callbacks=[tensorboard_callback_wide_2048],
    batch_size=2048
)

Train on 135897 samples, validate on 33975 samples
Epoch 1/20
135897/135897 [==============================] - 19s 138us/sample - loss: 2.7283 - accuracy: 0.3068 - val_loss: 1.8270 - val_accuracy: 0.5076
Epoch 2/20
135897/135897 [==============================] - 19s 143us/sample - loss: 1.8363 - accuracy: 0.5072 - val_loss: 1.5929 - val_accuracy: 0.5712
Epoch 3/20
135897/135897 [==============================] - 19s 138us/sample - loss: 1.6687 - accuracy: 0.5512 - val_loss: 1.5202 - val_accuracy: 0.5872
Epoch 4/20
135897/135897 [==============================] - 16s 117us/sample - loss: 1.5917 - accuracy: 0.5711 - val_loss: 1.4716 - val_accuracy: 0.5986
Epoch 5/20
135897/135897 [==============================] - 16s 116us/sample - loss: 1.5401 - accuracy: 0.5840 - val_loss: 1.4466 - val_accuracy: 0.6052
Epoch 6/20
135897/135897 [==============================] - 15s 114us/sample - loss: 1.5030 - accuracy: 0.5925 - val_loss: 1.4228 - val_accuracy: 0.6087
Epoch 7/20
135897/135897 [=====

In [10]:
x_txt_train.shape

(135897, 300)

In [29]:
img_sscaler = StandardScaler()
img_sscaler.fit(x_img_train)

x_img_train_ss = img_sscaler.transform(x_img_train)
x_img_val_ss = img_sscaler.transform(x_img_val)
x_img_test_ss = img_sscaler.transform(x_img_test)

txt_sscaler = StandardScaler()
txt_sscaler.fit(x_txt_train_modified)

x_txt_train_modified_ss = txt_sscaler.transform(x_txt_train_modified)
x_txt_val_ss = txt_sscaler.transform(x_txt_val)
x_txt_test_ss = txt_sscaler.transform(x_txt_test)

In [32]:
model_wide_2048_ss = models.get_model_custom_lr_wide()
log_dir_wide_2048 ="logs/zeros_replaced/wide_ss_2048_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback_wide_2048 = TensorBoard(log_dir=log_dir_wide_2048, histogram_freq=1)

model_wide_2048_ss.fit(
    [x_img_train_ss, x_txt_train_modified_ss],
    y_train,
    validation_data=([x_img_val_ss, x_txt_val_ss], y_val),
    epochs=20,
    callbacks=[tensorboard_callback_wide_2048],
    batch_size=2048
)

Train on 135897 samples, validate on 33975 samples
Epoch 1/20
135897/135897 [==============================] - 19s 142us/sample - loss: 2.4769 - accuracy: 0.3750 - val_loss: 1.6226 - val_accuracy: 0.5785
Epoch 2/20
135897/135897 [==============================] - 12s 85us/sample - loss: 1.7075 - accuracy: 0.5500 - val_loss: 1.4802 - val_accuracy: 0.6059
Epoch 3/20
135897/135897 [==============================] - 13s 95us/sample - loss: 1.5796 - accuracy: 0.5795 - val_loss: 1.4316 - val_accuracy: 0.6140
Epoch 4/20
135897/135897 [==============================] - 12s 90us/sample - loss: 1.5110 - accuracy: 0.5942 - val_loss: 1.3979 - val_accuracy: 0.6220
Epoch 5/20
135897/135897 [==============================] - 17s 127us/sample - loss: 1.4615 - accuracy: 0.6062 - val_loss: 1.3776 - val_accuracy: 0.6226
Epoch 6/20
135897/135897 [==============================] - 14s 106us/sample - loss: 1.4208 - accuracy: 0.6142 - val_loss: 1.3681 - val_accuracy: 0.6241
Epoch 7/20
135897/135897 [========

In [33]:
model_wide_4096_ss = models.get_model_custom_lr_wide()
log_dir_wide_4096 ="logs/zeros_replaced/wide_ss_4096_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback_wide_4096 = TensorBoard(log_dir=log_dir_wide_4096, histogram_freq=1)

model_wide_4096_ss.fit(
    [x_img_train_ss, x_txt_train_modified_ss],
    y_train,
    validation_data=([x_img_val_ss, x_txt_val_ss], y_val),
    epochs=30,
    callbacks=[tensorboard_callback_wide_2048],
    batch_size=4096
)

Train on 135897 samples, validate on 33975 samples
Epoch 1/30
135897/135897 [==============================] - 14s 103us/sample - loss: 2.8285 - accuracy: 0.3000 - val_loss: 1.8466 - val_accuracy: 0.5282
Epoch 2/30
135897/135897 [==============================] - 12s 88us/sample - loss: 1.8755 - accuracy: 0.5119 - val_loss: 1.5580 - val_accuracy: 0.5903
Epoch 3/30
135897/135897 [==============================] - 13s 95us/sample - loss: 1.6700 - accuracy: 0.5591 - val_loss: 1.4778 - val_accuracy: 0.6028
Epoch 4/30
135897/135897 [==============================] - 14s 102us/sample - loss: 1.5847 - accuracy: 0.5782 - val_loss: 1.4386 - val_accuracy: 0.6106
Epoch 5/30
135897/135897 [==============================] - 15s 107us/sample - loss: 1.5267 - accuracy: 0.5911 - val_loss: 1.4080 - val_accuracy: 0.6165
Epoch 6/30
135897/135897 [==============================] - 15s 110us/sample - loss: 1.4843 - accuracy: 0.6010 - val_loss: 1.3934 - val_accuracy: 0.6200
Epoch 7/30
135897/135897 [=======

In [43]:
zero_indicies = np.where(~x_txt_train_modified.any(axis=1))[0]
zero_indicies.shape

(17191,)

In [44]:
new_zeros=model_img_to_txt.predict(x_img_train[zero_indicies])

In [46]:
np.where(~new_zeros.any(axis=1))[0].shape

(17191,)